# Laboratorio 4
## Universidad del Valle de Guatemala <br> Facultad de Ingeniería
#### Departamento de Ciencias de la Computación <br> Data Science - Sección 10

In [28]:
# Importar librerías
import pandas as pd
import emoji
import nltk
import re


In [29]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#### 2. Cargar el archivo de datos.

In [30]:
# 1. Ya se descargo el archivo train.csv
data_frame = pd.read_csv('./data/train.csv')
data_frame

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


Se puede apreciar que se tienen las siguientes variables junto con su descripción y estado:

- id: Este sería el identificador del texto
- keyword: Palabra clave de como "catagolaron" el tweet
- location: El lugar en donde se publico el tewt
- text: El texto del mismo tweet publicado
- target: Clasificacion del tweet si es un desastre real o no

Se puede apreciar que se pueden encontrar datos vacios en keyword y en location.

#### 3. Limpie y preprocese los datos (Describir de forma detallada las actividades de preprocesamiento que se llevó a cabo)

In [31]:
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

In [32]:
stop_words = set(['the', 'a', 'an', 'in', 'at', 'is', 'it', 'this', 'that', 'on', 'was', 'by', 'for', 'with', 'as', 'and', 'or', 'to', 'from', 'of'])

def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return ' '.join(filtered_text)

In [33]:
# Convertir el texto en minúsculas
data_frame['text'] = data_frame['text'].str.lower()

# Quitar caracteres especiales “#”,”@” o los apóstrofes
data_frame['text'] = data_frame['text'].str.replace(r'[#@\'"]', '', regex=True)

# Quitar urls
data_frame['text'] = data_frame['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', regex=True)

# Eliminar signos de puntuacion
data_frame['text'] = data_frame['text'].str.replace(r'[^\w\s]', '', regex=True)

In [34]:
# Revisar si hay emojis en el texto y luego quitarlos
# data_frame['has_emoji'] = data_frame['text'].apply(has_emoji)
# data_frame['text'] = data_frame['text'].apply(remove_emojis)

data_frame['text'] = data_frame['text'].apply(remove_emojis)

In [37]:
# Quitar los artículos, preposiciones y conjunciones (stopwords)
data_frame['text'] = data_frame['text'].apply(remove_stopwords)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/cristian/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


#### 4. Obtener frecuencia de las palabras. (Tanto de los tweets de desastres como de los que no).

##### ¿Qué palabras cree que le servirán para hacer un mejor modelo de clasificación?

...

##### ¿Vale la pena explorar bigramas para analizar contexto?

...

#### 5. Realizar un análisis exploratorio de los datos para entrenderlos mejor, documente todos los análisis.

In [ ]:
# type here

#### 6. Elabore una función en el que el usuario ingrese un tweet y el sistema lo clasifica en desastre o no.

In [ ]:
# type here